# Music Generation - Siddharth Shah, Ian Pompliano

Artificial Neural Networks and Deep Learning Final Project (Fall 2023)

**Part 1: Collect and process data from midi files.**

In [26]:
import requests
import os
from bs4 import BeautifulSoup

# list of composers
composers = ["albeniz", "bach", "balak", "beeth", "borodin", "brahms", "burgm",
             "chopin", "clementi", "debuss", "godowsky", "grana", "grieg", "haydn",
             "liszt", "mendelssohn", "moszkowski", "mozart", "muss", "rach", "ravel",
             "schub", "schum", "sinding", "tschai"]

baseURL = 'http://www.piano-midi.de/'

# create a directory to save downloaded MIDI files
if not os.path.exists('midiFiles'):
    os.makedirs('midiFiles')

for composer in composers:
    # create URL using respective composer
    url = baseURL + composer + '.htm'
    response = requests.get(url)

    soup = BeautifulSoup(response.content, 'html.parser')

    # Find MIDI links on the composer's page
    midiLinks = soup.find_all('a', href=True)

    for link in midiLinks:
        # removes duplicate midi files (special format0)
        if link['href'].endswith('.mid') and '_format0' not in link['href']:
            midiURL = baseURL + link['href']
            fileName = f"midiFiles/{composer}_{link['href'].split('/')[-1]}"

            # download midi file and save it to  directory
            with open(fileName, 'wb') as midiFile:
                midiResponse = requests.get(midiURL)
                if midiResponse.status_code == 200:
                    midiFile.write(midiResponse.content)

